In [31]:
# Import Libraries
import pandas as pd
from datetime import datetime 
from unicodedata import normalize


In [32]:
# read in data 
df= pd.read_csv('cluster_raw.csv')

In [33]:
# view first 5 rows 
df.tail()

,tarikh,nama_kluster,kategori_kluster,negeri,daerah,jumlah_kes,jumlah_disaring
1535,2021-07-20,Jalan Waterfront,Tempat Kerja,Negeri Sembilan,Port Dickson,42,116
1536,2021-07-20,Pasar Besar Dungun,Tempat Kerja,Terengganu,Dungun,51,159
1537,2021-07-20,Kampung Pukak,Komuniti,Sabah,Tuaran,17,56
1538,2021-07-20,Jalan Bukit Chagar,Kumpulan Berisiko Tinggi,Johor,Johor Bahru,61,113
1539,2021-07-20,Jalan Berhala Merah,Tempat Kerja,WP Kuala Lumpur,Lembah Pantai,12,20


In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1540 entries, 0 to 1539
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   tarikh            1540 non-null   object
 1   nama_kluster      1540 non-null   object
 2   kategori_kluster  1540 non-null   object
 3   negeri            1540 non-null   object
 4   daerah            1540 non-null   object
 5   jumlah_kes        1540 non-null   object
 6   jumlah_disaring   1540 non-null   object
dtypes: object(7)
memory usage: 84.3+ KB


In [35]:
# obtain info on dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1540 entries, 0 to 1539
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   tarikh            1540 non-null   object
 1   nama_kluster      1540 non-null   object
 2   kategori_kluster  1540 non-null   object
 3   negeri            1540 non-null   object
 4   daerah            1540 non-null   object
 5   jumlah_kes        1540 non-null   object
 6   jumlah_disaring   1540 non-null   object
dtypes: object(7)
memory usage: 84.3+ KB


In [36]:
# remove trailing spaces
df = df.apply(lambda x: x.str.strip())

In [37]:
# convert column to datetime pandas
df['tarikh'] = pd.to_datetime(df['tarikh'])

In [38]:
# remove any empty spaces in columns with numerical values
df['jumlah_kes'] = df['jumlah_kes'].str.replace(r'\D+', '',regex=True)
df['jumlah_disaring'] = df['jumlah_disaring'].str.replace(r'\D+', '',regex=True)



In [39]:
# change columns with numbers to numerical values
df['jumlah_kes'] = pd.to_numeric(df['jumlah_kes'])
df['jumlah_disaring'] = pd.to_numeric(df['jumlah_disaring'])


In [40]:
# remove special characters in column
df['kategori_kluster'] = df['kategori_kluster'].map(lambda s: normalize('NFKD', s))

In [41]:
# clean mistakes in spelling
df['kategori_kluster'].replace(['Komunit'], 'Komuniti', regex=False, inplace=True )


In [42]:
# group same type of data
df.loc[df['kategori_kluster'].str.contains('Berisiko',case=False), 'kategori_kluster'] = 'Kumpulan Berisiko Tinggi'

# note : All the  various education clusters are grouped into one 
df.loc[df['kategori_kluster'].str.contains('Pendidikan',case=False), 'kategori_kluster'] = 'Pendidikan'

In [43]:
# view number of categories
df['kategori_kluster'].value_counts()

Tempat Kerja                878
Komuniti                    416
Keagamaan                    87
Kumpulan Berisiko Tinggi     66
Pendidikan                   54
Pusat Tahanan                37
Import                        2
Name: kategori_kluster, dtype: int64

In [44]:
df['negeri'].value_counts()

Selangor                                                       308
Johor                                                          259
Kelantan                                                       139
Sarawak                                                        131
WP Kuala Lumpur                                                127
Sabah                                                           96
Pulau Pinang                                                    91
Pahang                                                          68
Melaka                                                          64
Negeri Sembilan                                                 63
Kedah                                                           58
Perak                                                           58
Terengganu                                                      35
WP Labuan                                                        6
WP Kuala Lumpur dan Selangor                                  

In [45]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1540 entries, 0 to 1539
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   tarikh            1540 non-null   datetime64[ns]
 1   nama_kluster      1540 non-null   object        
 2   kategori_kluster  1540 non-null   object        
 3   negeri            1540 non-null   object        
 4   daerah            1540 non-null   object        
 5   jumlah_kes        1540 non-null   int64         
 6   jumlah_disaring   1540 non-null   int64         
dtypes: datetime64[ns](1), int64(2), object(4)
memory usage: 84.3+ KB


In [46]:
df.to_csv('cluster_processed.csv', index=False)

In [47]:

def loc_df(col):
    ''' this function returns a new df with the negeri or district column values seperated'''
    
    total_loc = []
    tarikh = []
    nama_kluster = []
    kategori_kluster = []
    negeri_sep = []
    for index, row in df.iterrows():
        locations = row[col].split(' dan')

        for l in locations : 
                # removing the , from the string and removing the trailing white spaces'
                loc_w_space = [x.strip() for x in l.split(',')]
                # removing locations which are just ' '
                clean_location = [e for e in loc_w_space if e.strip()]

                for i in clean_location:
                    # appending all values into seperate lists
                    total_loc.append(i)
                    tarikh.append(row['tarikh']) 
                    nama_kluster.append(row['nama_kluster'])
                    kategori_kluster.append(row['kategori_kluster'])

    #create new df  specific for the seperated column data
    df_new = pd.DataFrame({'tarikh':tarikh,'nama_kluster':nama_kluster, 'kategori_kluster': kategori_kluster, col : total_loc })
    return (df_new)


In [48]:
df_negeri = loc_df('negeri')

In [49]:
df_negeri

,tarikh,nama_kluster,kategori_kluster,negeri
0,2021-05-08,Kampung Kebun Sayur,Tempat Kerja,Johor
1,2021-05-08,Jalan Batu Maung,Pendidikan,Pulau Pinang
2,2021-05-08,Jalan Mawar 20,Keagamaan,Johor
3,2021-05-08,Keramat,Komuniti,Sabah
4,2021-05-08,Rantau Siantan,Komuniti,Pahang
...,...,...,...,...
1573,2021-07-20,Jalan Waterfront,Tempat Kerja,Negeri Sembilan
1574,2021-07-20,Pasar Besar Dungun,Tempat Kerja,Terengganu
1575,2021-07-20,Kampung Pukak,Komuniti,Sabah
1576,2021-07-20,Jalan Bukit Chagar,Kumpulan Berisiko Tinggi,Johor


In [50]:
#1310 rows × 4 columns

In [51]:
df_negeri['negeri'].value_counts()

Selangor           327
Johor              261
Kelantan           142
WP Kuala Lumpur    141
Sarawak            131
Sabah               97
Pulau Pinang        94
Pahang              73
Melaka              66
Negeri Sembilan     64
Perak               64
Kedah               60
Terengganu          39
WP Putrajaya        11
WP Labuan            7
Perlis               1
Name: negeri, dtype: int64

In [52]:
df_daerah = loc_df('daerah')

In [53]:
df_daerah

,tarikh,nama_kluster,kategori_kluster,daerah
0,2021-05-08,Kampung Kebun Sayur,Tempat Kerja,Muar
1,2021-05-08,Jalan Batu Maung,Pendidikan,Barat Daya
2,2021-05-08,Jalan Mawar 20,Keagamaan,Johor Bahru
3,2021-05-08,Keramat,Komuniti,Semporna
4,2021-05-08,Rantau Siantan,Komuniti,Temerloh
...,...,...,...,...
2263,2021-07-20,Jalan Waterfront,Tempat Kerja,Port Dickson
2264,2021-07-20,Pasar Besar Dungun,Tempat Kerja,Dungun
2265,2021-07-20,Kampung Pukak,Komuniti,Tuaran
2266,2021-07-20,Jalan Bukit Chagar,Kumpulan Berisiko Tinggi,Johor Bahru


In [54]:
# 1903 rows × 4 columns

In [55]:
df_daerah['daerah'].value_counts()

Johor Bahru             117
Petaling                 92
Klang                    91
Hulu Langat              75
Gombak                   60
                       ... 
Telupid                   1
Sebarang Perai Utara      1
Lubok Antu                1
Maradong                  1
Kangar                    1
Name: daerah, Length: 165, dtype: int64

In [56]:
df_negeri.to_csv('cluster_negeri.csv', index=False)

In [57]:
df_daerah.to_csv('cluster_daerah.csv', index=False)